## TP6

Una empresa de transporte quiere monitorear la actividad de sus conductores durante el manejo Para ello se propone implementar
un detector de somnolencia en tiempo real tomando como datos de entrada frames provenientes de una cámara que apunta al rosto
del conductor

1. Implementar un detector Haar de caras (vista frontal)

2. En esa ROI detectar los ojos (Se puede usar otro detector Haar

3. Mediante alguna de las técnicas vistas en clase detectar si ambos ojos están abiertos o cerrados y mostrar el estado de la detección sobre el frame
